In [70]:
import pandas as pd
import re
from dateutil import parser

In [113]:
with open('../data/events.csv') as f:
    data = pd.read_csv(f)
    starttime = data['starttime']
    endtime = data['endtime']

In [142]:
def sub_pattern(time_list, compiled_pats, is_endtime=False):
    collector = list()
    
    for st in time_list:
        if not pd.isnull(st) and st != '':
            check = list(map(lambda x: x.sub(r'\1', st) if x.match(st) else None, pats))
            if any(check):
                subbed = next(item for item in check if item is not None)
                collector.append(subbed)
            else:
                if len(compiled_pats) == 1:
                    collector.append(None)
                else:
                    collector.append('allday')
        else:
            if not is_endtime:
                if len(compiled_pats) == 1:
                    collector.append(None)
                else:
                    collector.append('allday')
            else:
                collector.append(None)
            
    return collector

# TODO
def cleansing(subbed_list):
    artifacts_removed = list(map(lambda x: x.replace(' ', '').replace('.', '').lower(), subbed_list))
    dt_list = [parser.parse(elem).time() if elem != 'allday' else elem for elem in artifacts_removed]
    
    return(dt_list)

In [144]:
# pattern for starttime and endtime 
patterns = [
    r'^[a-zA-Z]+, [a-zA-Z]+ [0-9]{1,2}, [0-9]{4} - (.*)',
    r'^([0-9]{1,2}:[0-9]{1,2}\s?[apmAPM\.]{0,4}).*',
    r'^([0-9]{1,2}\s?[apmAPM\.]{1,4}).*',
]

# compile to put as argument
pats = [re.compile(pat) for pat in patterns]

starttime_subbed = sub_pattern(starttime, compiled_pats=pats)
endtime_subbed = sub_pattern(endtime, compiled_pats=pats, is_endtime=True)

# pattern for extracting endtime out of starttime
patterns = [
    r'^[0-9]{1,2}:[0-9]{1,2}\s?[apmAPM\.]{0,4}\s?-\s?([0-9]{1,2}:[0-9]{1,2}\s?[apmAPM\.]{0,4})'
]

# compile to put as argument
pats = [re.compile(pat) for pat in patterns]

endtime_in_starttime_subbed = sub_pattern(starttime, compiled_pats=pats)

for i, j, k in zip(starttime_subbed, endtime_subbed, endtime_in_starttime_subbed):
    print(i,j,k)
    
# TODO
# if endtime is nan but starttime is not, endtime = starttime + 1 hour
# if endtime is nan but starttime is All day, endtime = All day
# if endtime is nan and starttime is nan, endtime = null, starttime = null
# if endtime is nan but starttime is XX:XX - YY:YY, endtime = YY:YY //

11:30 AM 12:30 PM None
11:30 AM 12:30 PM None
11:30 AM 12:30 PM None
11:30 AM 12:30 PM None
11:30 AM 12:30 PM None
11:30 AM 12:30 PM None
11:30 AM 12:30 PM None
11:30 AM 12:30 PM None
11:30 AM 12:30 PM None
11:30 AM 12:30 PM None
7:30 PM 8:30 PM None
11:00 AM 1:00 PM None
12:00 PM 1:30 PM None
10:30 AM 1:00 PM None
4:00 PM 5:00 PM None
12:00 PM 1:30 PM None
3:00 PM 5:00 PM None
10:00 AM 4:00 PM None
12:00 PM 2:00 PM None
5:00 PM 9:00 PM None
allday allday None
7:00 PM 9:30 PM None
12:00 PM 1:00 PM None
4:30 PM 5:30 PM None
allday allday None
allday allday None
allday allday None
allday allday None
allday allday None
allday allday None
12:00 PM 2:00 PM None
allday allday None
allday allday None
allday allday None
allday allday None
11:00 AM 4:00 PM None
12:00 PM 2:00 PM None
allday allday None
allday allday None
allday allday None
allday allday None
allday allday None
12:00 PM 2:00 PM None
allday allday None
allday allday None
allday allday None
12:00 PM 2:00 PM None
allday allday None
